Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en
las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el
estudio de grabación independientemente del número de tomas que se graben. No es
posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de
manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los
datos son:

Número de actores: 10

Número de tomas : 30

Actores/Tomas : https://bit.ly/36D8IuK

- 1 indica que el actor participa en la toma
- 0 en caso contrario

In [1]:
import random
from typing import List, Set, Tuple, Union, Optional, Literal

MATRIZ = [
    # Actores                       Tomas
    #1  2  3  4  5  6  7  8  9 10
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],  # 01
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],  # 02
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],  # 03
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],  # 04
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],  # 05
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],  # 06
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],  # 07
    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],  # 08
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],  # 09
    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],  # 10
    [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],  # 11
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],  # 12
    [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],  # 13
    [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],  # 14
    [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],  # 15
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],  # 16
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # 17
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],  # 18
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # 19
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],  # 20
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],  # 21
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],  # 22
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # 23
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],  # 24
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],  # 25
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],  # 26
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],  # 27
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # 28
    [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],  # 29
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # 30
]
outputs = list()

In [2]:
SolutionType = List[int]
NumericType = Union[float, int]
MAX_TOMAS_POR_DIA = 6

Modelo propuesto:
```
            T0, T1, T2, T3, ..., Tn
Solucion = [D0, D0, D2, D1, ..., Dm]
```
Donde el arreglo solución tendra por indices el identificativo de la toma
correspondiente, y el dia que corresponde a esa toma como valor dentro del
arreglo.

Partiremos de una solucion glotona o ambiciosa donde cada dia tendra las 6
tomas requeridas:

$$
\left\lceil N /6 \right\rceil
$$



In [3]:
TOMAS = [
    {i for i, val in enumerate(MATRIZ[j]) if val == 1}
    for j in range(len(MATRIZ))
]

def costo_total(solucion: SolutionType, tomas: List[Set[int]]) -> int:
    actores_por_dia = [set() for _ in range(max(solucion) + 1)]
    for toma, dia in enumerate(solucion):
        actores_por_dia[dia].update(tomas[toma])
    return sum(len(s) for s in actores_por_dia)

def generar_solucion_aleatoria(num_tomas: int):
    return random.sample(list(range(num_tomas)), num_tomas)

def normalizar_solucion(solucion: SolutionType):
    dias_usados = sorted(list(set(solucion)))
    mapa_dias = {dia_viejo: nuevo_indice for nuevo_indice, dia_viejo in enumerate(dias_usados)}
    solucion_normalizada = [mapa_dias[dia] for dia in solucion]
    return solucion_normalizada

def convertir_solucion(solucion):
    dias = [set() for _ in range(max(solucion) + 1)]
    for toma, dia in enumerate(solucion):
        dias[dia].add(toma)
    return tuple(dias)


In [4]:
def generar_vecina(
        tomas: List[Set[int]],
        solucion_inicial: Optional[SolutionType] = None,
        max_tomas_por_dia: int = MAX_TOMAS_POR_DIA
) -> Tuple[SolutionType, NumericType]:
    if solucion_inicial is None:
        mejor_solucion = generar_solucion_aleatoria(len(tomas))
    else:
        mejor_solucion = list(solucion_inicial)

    mejor_costo = costo_total(mejor_solucion, tomas)

    dias_activos = set(mejor_solucion)
    max_dia_activos = max(dias_activos)

    # vemos si representa una mejora aumentar un dia mas
    posibles_dias = list(range(min(len(mejor_solucion), max_dia_activos + 2)))
    for toma in range(len(mejor_solucion)):
        dia_origen = mejor_solucion[toma]

        for dia_destino in posibles_dias:
            if dia_origen == dia_destino:
                continue
            tomas_destino = mejor_solucion.count(dia_destino)

            if tomas_destino >= max_tomas_por_dia:
                continue

            vecino = list(mejor_solucion)
            vecino[toma] = dia_destino
            costo_nuevo = costo_total(vecino, tomas)

            if costo_nuevo <= mejor_costo:
                mejor_costo = costo_nuevo
                mejor_solucion = vecino

    mejor_solucion = normalizar_solucion(mejor_solucion)
    return mejor_solucion, mejor_costo

solucion, costo = generar_vecina(TOMAS, max_tomas_por_dia=MAX_TOMAS_POR_DIA)
print(solucion, costo)

[6, 6, 9, 2, 7, 6, 6, 1, 6, 1, 6, 3, 5, 3, 2, 0, 8, 3, 8, 4, 7, 3, 4, 3, 3, 4, 5, 5, 5, 5] 40
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [5]:
def busqueda_local(
        tomas: List[Set[int]],
        solucion_inicial: Optional[SolutionType] = None,
        max_tomas_por_dia: int = MAX_TOMAS_POR_DIA
) -> Tuple[SolutionType, NumericType]:
    if solucion_inicial is None:
        solucion_referencia = generar_solucion_aleatoria(len(tomas))
    else:
        solucion_referencia = list(solucion_inicial)
    mejor_costo = costo_total(solucion_referencia, tomas)

    while True:
        nueva_solucion, costo_vecina = generar_vecina(tomas, solucion_referencia, max_tomas_por_dia)
        if costo_vecina < mejor_costo:
            solucion_referencia = nueva_solucion  #Guarda la mejor solución encontrada
            mejor_costo = costo_vecina
        else:
            return solucion_referencia, mejor_costo

solucion, costo = busqueda_local(
    TOMAS,
    solucion_inicial=solucion,
    max_tomas_por_dia=MAX_TOMAS_POR_DIA
)
print(solucion, costo)
print(set(solucion))

[5, 3, 5, 1, 1, 5, 5, 0, 1, 0, 5, 2, 4, 2, 5, 1, 3, 2, 3, 3, 0, 2, 3, 2, 1, 3, 4, 4, 2, 4] 32
{0, 1, 2, 3, 4, 5}


In [6]:
def mover_una_toma(
        solucion: SolutionType,
        max_tomas: int = MAX_TOMAS_POR_DIA
) -> SolutionType:
    solucion_vecina = list(solucion)
    num_tomas = len(solucion_vecina)
    while True: # un while podría dejar en un bucle infinito
        toma_random = random.randint(0, num_tomas - 1)
        dia_actual = solucion_vecina[toma_random]
        max_dia = max(solucion_vecina)
        dia_destino = random.randint(0, max_dia + 2)
        if dia_actual != dia_destino and solucion_vecina.count(dia_destino) < max_tomas:
            solucion_vecina[toma_random] = dia_destino
            return solucion_vecina

def swap(solucion: SolutionType) -> SolutionType:
    solucion_vecina = list(solucion)
    num_tomas = len(solucion_vecina)
    while True:
        toma_a = random.randint(0, num_tomas - 1)
        toma_b = random.randint(0, num_tomas - 1)
        dia_a = solucion_vecina[toma_a]
        dia_b = solucion_vecina[toma_b]
        if dia_a != dia_b:
            solucion_vecina[toma_a] = dia_b
            solucion_vecina[toma_b] = dia_a
            return solucion_vecina

def fusionar_dias(
        solucion: SolutionType,
        intensidad: int,
        max_tomas: int = MAX_TOMAS_POR_DIA,
        max_intentos: int = 50
) -> SolutionType:
    solucion_vecina = list(solucion)
    num_tomas = len(solucion_vecina)
    movimientos_exitosos = 0
    intentos = 0
    while movimientos_exitosos < intensidad + 1 and intentos < max_intentos:
        toma_random = random.randint(0, num_tomas - 1)
        dia_destino = random.randint(0, max(solucion_vecina) + 2)
        if solucion_vecina[toma_random] != dia_destino and solucion_vecina.count(dia_destino) < max_tomas:
            solucion_vecina[toma_random] = dia_destino
            movimientos_exitosos += 1
        intentos += 1
    return solucion_vecina


def perturbar(
        solucion: SolutionType,
        intensidad: Literal[0, 1, 2, 3],
        max_tomas: int = MAX_TOMAS_POR_DIA,
) -> SolutionType:
    if intensidad > 3:
        raise ValueError('Intensidad > 3')
    if intensidad == 0:
        return mover_una_toma(solucion, max_tomas)
    elif intensidad == 1:
        return swap(solucion)
    return fusionar_dias(solucion, intensidad, max_tomas)

In [7]:
def busqueda_local_con_entornos_variables(
        tomas: List[Set[int]],
        solucion_inicial: Optional[SolutionType] = None,
        max_tomas: int = MAX_TOMAS_POR_DIA,
        max_iter: int = 1000,
        verbose: bool = False
) -> Tuple[SolutionType, NumericType]:
    if solucion_inicial:
        solucion_actual = list(solucion_inicial)
    else:
        solucion_actual = generar_solucion_aleatoria(len(tomas))
    mejor_costo = float('inf')
    k_max = 2
    iteracion = 0

    while iteracion <= max_iter:
        k = 0
        while k <= k_max and iteracion <= max_iter:
            iteracion += 1
            vecino = perturbar(solucion_actual, k)
            vecino_mejorado, costo_vecino = busqueda_local(
                 tomas, vecino, max_tomas
            )
            if costo_vecino < mejor_costo:
                solucion_actual = vecino_mejorado
                mejor_costo = costo_vecino
                k = 0
                if verbose:
                    print(
                        "En la iteración ", iteracion,
                        ", la mejor solución encontrada es:", solucion_actual
                    )
                    print("Costo: ", costo_vecino)
            else:
                k += 1
    return solucion_actual, mejor_costo

In [8]:
print(busqueda_local_con_entornos_variables(TOMAS, solucion, max_iter=1000))

([1, 1, 0, 0, 3, 4, 4, 0, 4, 3, 3, 3, 4, 2, 0, 4, 2, 2, 2, 1, 0, 1, 2, 2, 4, 3, 3, 1, 0, 1], 27)


In [9]:
def resolver_vns_multi_start(
        tomas: List[Set[int]],
        intentos: int = 3,
        max_iter: int = 1000,
        max_tomas: int = MAX_TOMAS_POR_DIA,
) -> Tuple[SolutionType, NumericType]:
    mejor_solucion_global = None
    mejor_costo_global = float('inf')

    for i in range(intentos):

        semilla = random.sample(list(range(30)), 30)

        solucion_candidata, costo_candidato = busqueda_local_con_entornos_variables(
            tomas=tomas,
            solucion_inicial=semilla,
            max_tomas=max_tomas,
            max_iter=max_iter,
            verbose=False
        )
        if costo_candidato < mejor_costo_global:
            mejor_costo_global = costo_candidato
            mejor_solucion_global = solucion_candidata

    return mejor_solucion_global, mejor_costo_global

In [10]:
failed, asserted = 0, 0
max_iters = 100
for _ in range(1, max_iters+1):
    solucion, costo = resolver_vns_multi_start(TOMAS, 2, 2000)
    print(_, solucion, costo)
    if costo <= 27:
        asserted += 1
        print(f"Se obtuvo el minimo esperado {asserted/_}")
        sol = convertir_solucion(solucion)
        if sol not in outputs:
            outputs.append(convertir_solucion(solucion))
    else:
        failed += 1
        print(f"No se obtuvo el minimo esperado {asserted/_}")

print(asserted/max_iters, failed/max_iters)

1 [2, 2, 4, 4, 4, 4, 4, 1, 0, 1, 1, 0, 2, 3, 4, 0, 3, 3, 3, 2, 1, 0, 3, 3, 0, 1, 2, 0, 1, 2] 27
Se obtuvo el minimo esperado 1.0



KeyboardInterrupt



In [ ]:
print(outputs)